In [5]:
#@title pip installs
!pip install pandas

In [4]:
#@title Imports and setting up Access Layer Calls
import os
import requests

import pandas as pd

import pprint
pp = pprint.PrettyPrinter(indent=4)

_SERVICE_URL = 'https://development-admin-app-backend-2jv3ndpkoa-uc.a.run.app/'
headers = {
    'Accept': 'application/json',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36',
}

email = ""  # @param {type:"string"}
password= ""  # @param {type:"string"}
query = """mutation {
  createToken(email: "%s", password: "%s") {
    accessToken
    tokenType
    __typename
  }
}
""" % (email, password)

access_layer_url = os.path.join(_SERVICE_URL, 'graphql')
response = requests.post(
    access_layer_url,
    headers=headers,
    json={'query': query})
jwt_token = response.json().get("data", {}).get('createToken', {}).get('accessToken')
headers['Authorization'] = f'Bearer {jwt_token}'


## Example
Below is an example Access layer GraphQL Query for paginated biomarker results that have two variables added into the example --`skip` and `limit`.

Additional example queries can be found within this [directory](https://github.com/Function-Health/admin-app-fe-next/tree/dev/src/apollo/queries) but note that these examples are based on Typescript and not Python.

Note that when changing the below queries the return JSON will also change. For example if you were to query the `symptoms` tables the return JSON strcture will look something like `{"data": {"biomarkerResultsPaginated": {...})`

A full list of Access Layer Schemas can be found [here](https://github.com/Function-Health/admin-app-backend/tree/dev/app/schemas)

In [ ]:
#@title Pull all patients from the Access Layer
skip = 0  # @param {type:"string"}
limit = 10  # @param {type:"string"}

query = """
{
  patients(skip: %s, limit: %s) {
    id
    lname
    biologicalSex
    dob
    ethnicOrigin
  }
}
""" % (skip, limit)

# Call Access Layer
response = requests.post(
    access_layer_url,
    headers=headers,
    json={'query': query})

access_layer_return = response.json()
pp.pprint(access_layer_return)
extract_access_layer_data = access_layer_return.get(
    "data", {}).get("patients", {})
access_layer_df = pd.DataFrame(extract_access_layer_data)

# Print the resulting dataframe
pp.pprint(access_layer_df)


In [ ]:
#@title Pull single patient from the Access Layer
patient_id = "ceda71be-5706-43cb-a5b3-8ed4f6aa275f" # @param {type:"string"}

query = """
query patients($filterOps: PatientFilterOps) {
        patients(filterOps: $filterOps) {
            id
            biologicalSex
            dob
            fname
            preferredName
            ethnicOrigin
            patientContactInfo {
                email
            }
            motivational {
                tenYearHealthGoals
            }
            femaleReproductiveHealth {
                currentMenstrualStatus
                currentlyPregnant
                birthControlType
            }
            patientBehaviorInfo{
                diet
                smokingFrequency
                alcoholFrequency
                physicallyActive
                smoker
                alcohol
            }
            medicalHx {
                currentMedications
                surgeries
                hospitalizations
                chronicConditions
                chronicConditionDiagnosisDates
            }
        }
    }
"""
variables = {
  "filterOps": {
      "id": patient_id
  }
}

# Call Access Layer
response = requests.post(
    access_layer_url,
    headers=headers,
    json={
        "query": query,
        "variables": variables
      })

access_layer_return = response.json()
pp.pprint(access_layer_return)
extract_access_layer_data = access_layer_return.get(
    "data", {}).get("patients", [])
access_layer_df = pd.DataFrame(extract_access_layer_data)

# Print the resulting dataframe
pp.pprint(access_layer_df)


In [ ]:
#@title Pull all Requisitions from the Access Layer
skip = 0  # @param {type:"string"}
limit = 10  # @param {type:"string"}
patient_id = "f4cdc35c-4608-4779-861d-1ec857801dd9" # @param {type:"string"}

query = """
query requisitions($filterOps: RequisitionFilterOps) {
        requisitions(filterOps: $filterOps) {
            id
            patientId
            allResultsAvailable
            reviewed
            dateTimeReviewed
            dateTimeConfirmed
            visits{
                id
                visitDate
                biomarkerResults {
                    questBiomarkerId
                    biomarkerName
                    testResult
                    measurementUnits
                    testResultOutOfRange
                    questReferenceRange
                }
            }
        }
    }
"""
variables = {
  "filterOps": {
      "patientId": patient_id
  }
}

# Call Access Layer
response = requests.post(
    access_layer_url,
    headers=headers,
    json={
        "query": query,
        "variables": variables
      })

access_layer_return = response.json()
pp.pprint(access_layer_return)
extract_access_layer_data = access_layer_return.get(
    "data", {}).get("requisitions", [])
access_layer_df = pd.DataFrame(extract_access_layer_data)

# Print the resulting dataframe
pp.pprint(access_layer_df)


In [ ]:
#@title Pull Recommendations by ID

recommendation_id = "6bc8603e-043e-4407-a188-3033f4300b41" # @param {type:"string"}

query = """
query recommendations($filterOps: RecommendationFilterOps) {
        recommendations(filterOps: $filterOps) {
            id
            outOfRangeType
            summaryDescription
            causesDescription
            symptomsDescription
            foodsToEatDescription
            foodsToAvoidDescription
            supplementsDescription
            selfCareDescription
            additionalTestsDescription
            followUpsDescription
        }
    }
"""
variables = {
  "filterOps": {
      "id": recommendation_id
  }
}

# Call Access Layer
response = requests.post(
    access_layer_url,
    headers=headers,
    json={
        "query": query,
        "variables": variables
      })

access_layer_return = response.json()
pp.pprint(access_layer_return)

extract_access_layer_data = access_layer_return.get(
    "data", {}).get("recommendations", [])
access_layer_df = pd.DataFrame(extract_access_layer_data)

# Print the resulting dataframe
pp.pprint(access_layer_df)


In [ ]:
#@title Pull all recommendations
skip = 0  # @param {type:"string"}
limit = 10  # @param {type:"string"}

query = """
{
  recommendations(skip: %s, limit: %s) {
    id
    summaryDescription
  }
}
""" % (skip, limit)

# Call Access Layer
response = requests.post(
    access_layer_url,
    headers=headers,
    json={'query': query})

access_layer_return = response.json()
pp.pprint(access_layer_return)
extract_access_layer_data = access_layer_return.get(
    "data", {}).get("recommendations", {})
access_layer_df = pd.DataFrame(extract_access_layer_data)

# Print the resulting dataframe
pp.pprint(access_layer_df)


In [ ]:
#@title Pull all Biomarker Results from the Access Layer
skip = 0  # @param {type:"string"}
limit = "1000"  # @param {type:"string"}

query = """
{
  biomarkerResults(skip: %s, limit: %s) {
    biomarkerName
    questReferenceRange
  }
}
""" % (skip, limit)

# Call Access Layer
response = requests.post(
    access_layer_url,
    headers=headers,
    json={'query': query})

access_layer_return = response.json()
pp.pprint(access_layer_return)
extract_access_layer_data = access_layer_return.get(
    "data", {}).get("biomarkerResults", {})
access_layer_df = pd.DataFrame(extract_access_layer_data)

# Print the resulting dataframe
pp.pprint(access_layer_df)
